In [2]:
#have used sklearn and ktrain modules, they need installation
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 
import pandas as pd
import pickle
import torch
import time
import tensorflow as tf
import ktrain
from ktrain import text
import random 

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast
device = torch.device("cuda")
#used train.csv and test.csv and dev.csv file from codalab
trainingfile = 'train.csv'
testingfile = 'test.csv'
validationfile = 'dev.csv'

In [4]:

trainingdata_S = pd.read_csv(trainingfile)
testingdata_S = pd.read_csv(testingfile)
validationdata_S = pd.read_csv(validationfile)

In [5]:
#remove columns
drop_labels = ['Confusing Reason1' 	,'Confusing Reason2', 	'Right Reason2']
trainingdata_S.drop(drop_labels, axis = 1, inplace= True)
testingdata_S.drop(drop_labels, axis = 1, inplace= True)
validationdata_S.drop(drop_labels, axis = 1, inplace= True)

In [7]:
correct_statement= []
incorrect_statement=[]
right_reason1= []
right_reason2 = []

for i in range(len(trainingdata_S['Correct Statement'])):
  correct_statement.append([trainingdata_S.iloc[i,0], 1])
  incorrect_statement.append([trainingdata_S.iloc[i,1], 0])
  right_reason1.append([trainingdata_S.iloc[i,2], 1])
  right_reason2.append([trainingdata_S.iloc[i,3], 1])

In [8]:
datalist = correct_statement + incorrect_statement + right_reason1 + right_reason2
random.shuffle(datalist)

In [9]:
correct_statement= []
incorrect_statement=[]
right_reason1= []
right_reason2 = []

for i in range(len(testingdata_S['Correct Statement'])):
  correct_statement.append([testingdata_S.iloc[i,0], 1])
  incorrect_statement.append([testingdata_S.iloc[i,1], 0])
  right_reason1.append([testingdata_S.iloc[i,2], 1])
  right_reason2.append([testingdata_S.iloc[i,3], 1])

In [10]:
testlist = correct_statement + incorrect_statement + right_reason1 + right_reason2
random.shuffle(testlist) 

In [11]:
correct_statement= []
incorrect_statement=[]
right_reason1= []
right_reason2 = []

for i in range(len(validationdata_S['Correct Statement'])):
  correct_statement.append([validationdata_S.iloc[i,0], 1])
  incorrect_statement.append([validationdata_S.iloc[i,1], 0])
  right_reason1.append([validationdata_S.iloc[i,2], 1])
  right_reason2.append([validationdata_S.iloc[i,3], 1])

In [14]:
validationlist = correct_statement + incorrect_statement + right_reason1 + right_reason2
random.shuffle(validationlist) 

In [15]:
columnlabels = ['Sentence', 'Values']
traindata_df = pd.DataFrame(datalist, columns = columnlabels)
testdata_df = pd.DataFrame(testlist, columns = columnlabels)
devdata_df = pd.DataFrame(validationlist, columns = columnlabels)

In [16]:
traindata_df.head()

,Sentence,Values
0,you need to be asleep to stay up,0
1,I drank some cactus from a thermos.,0
2,Medication is not sold at gas stations.,1
3,Audiences perform during a play.,0
4,new batteries provide power to make the flashl...,1


In [17]:
(X_t, Y_t) , (X_T, Y_T), preprocess = text.texts_from_df(train_df= traindata_df, 
text_column= 'Sentence', 
label_columns= 'Values',
val_df = devdata_df,
maxlen = 400,
preprocess_mode = 'bert')      

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [18]:
model = text.text_classifier(name='bert',train_data = (X,Y),preproc = preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [19]:
learner = ktrain.get_learner(model = model,train_data = (X,Y),val_data = (X_T, Y_T),batch_size = 6)

In [ ]:
learner.fit_onecycle(lr = 2e-2, epochs=1)